In [1]:
import numpy as np
import torch
from stork.constraints import MinMaxConstraint
from LPLDataset import SparseGenerator, LPLDataset
from LPLModel import LPLModel
from LPLGroup import LPLGroup, InputGroup
from LPLConnection import SymmetricSTDPConnection, LPLConnection, Connection

In [2]:
# Model Parameters
dt = 2e-3
time_window = 0.5
nb_time_steps = (int)(time_window/dt)
timestep_rmsprop_updates = nb_time_steps
batch_size = 1
nb_inputs = 500  
nb_exc_units = 100
nb_inh_units = 25
device = torch.device("cuda")
dtype = torch.float
epoch = 10
file_name = ""

In [3]:
data1 = np.loadtxt("spks/5hz_spikes_10000s_2.ras")
data2 = np.loadtxt("spks/5hz_spikes_100s_2.ras")


In [4]:
dataset1 = LPLDataset(data1, time_step=dt, time_window=time_window, device=device)
dataset2 = LPLDataset(data2, time_step=dt, time_window=time_window, device=device)
generator = SparseGenerator(nb_workers=0)

In [5]:
lpl_con_prob = 0.1
ii_con_prob  = 0.1
ie_con_prob  = 0.5
lpl_con_w    = 0.15
ii_con_w     = 0.4
ie_con_w     = 0.1

ei_con_w     = 0.5

## LPL Model

In [6]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

con1 = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    constraints=constraints1,
    lamda_=1.0,
    phi=1.0,
    lr=1e-2,
    delta_time=20e-3,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

con2 = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True,
        initial_weight=ei_con_w, 
        name="Con1", 
        target="exc"))

con3 = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Con2", 
        target="inh"))

con4 = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="STDPCon", 
        target="inh",
        constraints=constraints2,
        lr=1e-3,
        kappa=10))

model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

con1.set_evolve(True)
con4.set_evolve(True)



# Model summary

## Groups
input, (500,)
exc, (100,)
inh, (25,)

## Connections
LPLConnection(
  (src): InputGroup()
  (dst): LPLGroup()
  (op): Linear(in_features=500, out_features=100, bias=False)
)
Connection(
  (src): LPLGroup()
  (dst): LPLGroup()
  (op): Linear(in_features=100, out_features=25, bias=False)
)
Connection(
  (src): LPLGroup()
  (dst): LPLGroup()
  (op): Linear(in_features=25, out_features=25, bias=False)
)
SymmetricSTDPConnection(
  (src): LPLGroup()
  (dst): LPLGroup()
  (op): Linear(in_features=25, out_features=100, bias=False)
)


In [34]:
# model.stored = False
# model.map = {}
# model.store_out = {}
# model.store_weight = {}
model.stimulate(dataset1, epoch=1)

stimulate start!
   epoch 0 end!
   epoch 1 end!
   epoch 2 end!


In [35]:
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [con1.name]
model.configure_stored(list_neurons, list_con, "_10000s_lpl")
model.stimulate(dataset2)

stimulate start!
   epoch 0 end!
   out_50000s_2_lpl have stored!
   weight_50000s_2_lpl have stored!


In [39]:
# print
# (
#     con1.trace_post_mean, 
#     con1.trace_post_sigma2, 
#     con1.err
# )
# for i in con4.op.weight.data:
#     print(i)
print(con1.el_val_flt * (con1.trace_err_flt.T @ torch.ones_like(con1.src.out)))

tensor([[-3.9910e-10, -5.2004e-12, -1.0247e-08,  ..., -1.2910e-02,
         -5.1996e-03, -1.3283e-05],
        [-3.9539e-10, -5.1521e-12, -1.0152e-08,  ..., -1.2789e-02,
         -5.1482e-03, -1.3159e-05],
        [-8.4248e-10, -1.0978e-11, -2.1630e-08,  ..., -1.5172e-02,
         -5.1643e-03, -2.8037e-05],
        ...,
        [-8.6581e-10, -1.1282e-11, -2.2229e-08,  ..., -2.8038e-02,
         -1.1365e-02, -2.8816e-05],
        [-4.0047e-10, -5.2182e-12, -1.0282e-08,  ..., -1.2954e-02,
         -5.2185e-03, -1.3328e-05],
        [-8.7631e-10, -1.1419e-11, -2.2499e-08,  ..., -1.5808e-02,
         -5.1660e-03, -2.9163e-05]], device='cuda:0')


### Pred Off Model

In [ ]:

model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

con1 = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=60,
    constraints=None,
    lamda_=1.0,
    phi=0.0))

con2 = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True, 
        name="Con1", 
        target="exc"))

con3 = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Con2", 
        target="inh"))

con4 = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="STDPCon", 
        target="inh",
        constraints=constraints2))

model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

con1.set_evolve(True)
con4.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [con1.name]
model.configure_stored(list_neurons, list_con, file_name + "_pred_off")
model.stimulate(dataset2)

## Hebb Off Model

In [ ]:

model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

con1 = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=60,
    constraints=constraints1,
    lamda_=0.0,
    phi=1.0,
    lr=0.01))

con2 = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True, 
        name="Con1", 
        target="exc"))

con3 = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Con2", 
        target="inh"))

con4 = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="STDPCon", 
        target="inh",
        constraints=constraints2,
        lr=0.001))

model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

con1.set_evolve(True)
con4.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=3)
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [con1.name]
model.configure_stored(list_neurons, list_con, file_name + "_hebb_off")
model.stimulate(dataset2)

## Inhib off Model

In [ ]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))

# constraints1 = MinMaxConstraint(0.0, 1.0)

con1 = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=60,
    constraints=None,
    lamda_=1.0,
    phi=1.0))

model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

con1.set_evolve(True)

In [ ]:
model.stimulate(dataset2, epoch=epoch)
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [con1.name]
model.configure_stored(list_neurons, list_con, file_name + "_inhib_off")
model.stimulate(dataset2)

## Inhib Fixed Model

In [ ]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
# constraints1 = MinMaxConstraint(0.0, 1.0)


con1 = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=60,
    constraints=None,
    lamda_=1.0,
    phi=1.0))

con2 = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True, 
        name="Con1", 
        target="exc"))

con3 = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Con2", 
        target="inh"))

con4 = model.add_connection(
    Connection(
        inh_neurons, 
        exc_neurons, 
        gaussian=True,
        name="Con3", 
        target="inh",
        constraints=None))

model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

con1.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [con1.name]
model.configure_stored(list_neurons, list_con, file_name + "_inhib_fixed")
model.stimulate(dataset2)